In [ ]:
# import deep learning libraries
from tensorflow import keras
from tensorflow.keras.datasets import mnist

# import preprocessing
from sklearn.preprocessing import LabelBinarizer

# import metrics
from sklearn.metrics import classification_report

# import image display library
import matplotlib.pyplot as plt

In [ ]:
# load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# view the inputs
plt.figure()
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(str(y_train[i]))
    plt.imshow(x_train[i], cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
def transform(img):
    """Convert image to float32 data type and make range [0, 1]."""
    return img.astype('float32') / 255

# transform the inputs
transformed_x_train = transform(x_train)
transformed_x_test = transform(x_test)

# view the inputs
plt.figure()
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(str(y_train[i]))
    plt.imshow(transformed_x_train[i], cmap='gray', vmin=0, vmax=1)
plt.show()

In [ ]:
# convert labels to one hot encoding
encoder = LabelBinarizer()
transformed_y_train = encoder.fit_transform(y_train)
transformed_y_test = encoder.transform(y_test)

In [ ]:
def get_model():
    model = keras.Sequential()

    # input layer
    model.add(keras.layers.InputLayer(input_shape=(28, 28, 1)))

    # first convolution layer
    model.add(keras.layers.Conv2D(32, (5, 5), padding='same', kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    # second convolution layer
    model.add(keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # transition to fully connected layers
    model.add(keras.layers.Flatten())

    # first dense layer 
    model.add(keras.layers.Dense(64, kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.Dropout(0.5))

    # second dense layer
    model.add(keras.layers.Dense(64, kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.Dropout(0.5))

    # output layer
    model.add(keras.layers.Dense(10, kernel_initializer='he_uniform'))
    model.add(keras.layers.Softmax())
    return model

In [ ]:
# create a model
model = get_model()
print(model.summary())
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics='accuracy'
)

In [ ]:
# train the model
model.fit(transformed_x_train, transformed_y_train, batch_size=128, epochs=10, validation_data=(transformed_x_test, transformed_y_test))

In [ ]:
# evaluate the model
print(
    classification_report(
        model.predict(transformed_x_test).argmax(axis=1), 
        transformed_y_test.argmax(axis=1), 
        target_names=[str(x) for x in encoder.classes_]
    )
)

In [ ]:
# save the model
model.save('base_model.keras')

In [ ]:
# load the model
reloaded = keras.models.load_model('base_model.keras')

In [ ]:
# evaluate the model
print(
    classification_report(
        reloaded.predict(transformed_x_test).argmax(axis=1), 
        transformed_y_test.argmax(axis=1), 
        target_names=[str(x) for x in encoder.classes_]
    )
)